In [116]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.models import googlenet, GoogLeNet_Weights, resnet50, ResNet50_Weights
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from skimage import io

import os
import shutil
import PIL
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [101]:
TRAIN_ANNOTATIONS = 'data/train_ann.csv' # path to file with classes annotations for train images
TRAIN_PATH = 'data/train' # path to training dataset
TEST_PATH = 'data/test' # path to test dataset for preditions

In [102]:
annotations = pd.read_csv(TRAIN_ANNOTATIONS)

In [103]:
assert annotations['class'].nunique()==196 # check if all classes are in train

https://colab.research.google.com/drive/1nkuq29334tLTI5Zk2odCuuBdbPpuNjN2

https://colab.research.google.com/drive/1jbH8VCPhlv3isy-3Jij8_YvYRG4dHz_H#scrollTo=758f1aa7

https://github.com/isadrtdinov/intro-to-dl-hse/blob/2022-2023/homeworks-small/shw-02-cnn.ipynb

In [104]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [105]:
# Hyperparameters
IN_CHANNEL = 3
NUM_CLASSES = 196
LEARNING_RATE = 1e-3
BATCH_SIZE = 32
NUM_EPOCHS = 25

In [106]:
# Create dataloaders
class CarsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = PIL.Image.open(img_path)
        image.convert('RGB')
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [107]:
from torchvision.transforms import v2
transforms = v2.Compose([
    v2.ToImage(), # Convert to tensor, only needed if you had a PIL image
    v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
    # ...
    v2.RandomHorizontalFlip(),
    v2.RandomResizedCrop(size=(300, 300), antialias=True),  # Or Resize(antialias=True)
    # ...
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [108]:
train_dataset = CarsDataset(csv_file='data/train_ann.csv', 
                            root_dir='data',
                            transform=transforms)

In [109]:
train_set, val_set = torch.utils.data.random_split(train_dataset, [0.75, 0.25])
train_loader = DataLoader(dataset = train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset = val_set, batch_size=BATCH_SIZE, shuffle=True)

In [110]:
model = googlenet(weights=GoogLeNet_Weights.DEFAULT)
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [111]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [112]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions ==y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy of {round(float(num_correct)/float(num_samples)*100, 2)}')

    model.train()

In [113]:
# Train Network
for epoch in tqdm(range(NUM_EPOCHS)):
    losses = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda/MPS
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # Backward step
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
    print(f'Checking accuracy on Training set after Epoch: {epoch}')
    check_accuracy(train_loader, model)

    print(f'Checking accuracy on Validation set after Epoch: {epoch}')
    check_accuracy(val_loader, model)
        

  0%|          | 0/25 [00:00<?, ?it/s]

Checking accuracy on Training set after Epoch: 0
Got 370 / 6099 with accuracy of 6.07
Checking accuracy on Validation set after Epoch: 0
Got 101 / 2033 with accuracy of 4.97
Checking accuracy on Training set after Epoch: 1
Got 665 / 6099 with accuracy of 10.9
Checking accuracy on Validation set after Epoch: 1
Got 163 / 2033 with accuracy of 8.02
Checking accuracy on Training set after Epoch: 2
Got 1393 / 6099 with accuracy of 22.84
Checking accuracy on Validation set after Epoch: 2
Got 346 / 2033 with accuracy of 17.02
Checking accuracy on Training set after Epoch: 3
Got 2253 / 6099 with accuracy of 36.94
Checking accuracy on Validation set after Epoch: 3
Got 589 / 2033 with accuracy of 28.97
Checking accuracy on Training set after Epoch: 4
Got 2524 / 6099 with accuracy of 41.38
Checking accuracy on Validation set after Epoch: 4
Got 669 / 2033 with accuracy of 32.91
Checking accuracy on Training set after Epoch: 5
Got 3406 / 6099 with accuracy of 55.85
Checking accuracy on Validation s

In [ ]:
# Make predictions on Test data



In [131]:
# Попробовать ResNet50 и доучивать последний слой
model_resnet = resnet50(weights=ResNet50_Weights.DEFAULT)

In [137]:
for name, param in model_resnet.named_parameters():
    if 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion_resnet = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet.parameters(), lr=0.001, momentum=0.9)
#optim.Adam(model.parameters(), lr=LEARNING_RATE)
model_resnet = model_resnet.to(device)

In [ ]:
# Train Network
for epoch in tqdm(range(50)):
    losses = []
    print(f"On epoch {epoch+50}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to Cuda/MPS
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_resnet(data)
        loss = criterion_resnet(scores, targets)

        losses.append(loss.item())

        # Backward step
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    if epoch % 5 == 0:    
        print(f'Checking accuracy on Training set after Epoch: {epoch+50}')
        check_accuracy(train_loader, model_resnet)

        print(f'Checking accuracy on Validation set after Epoch: {epoch+50}')
        check_accuracy(val_loader, model_resnet)

  0%|          | 0/50 [00:00<?, ?it/s]

On epoch 0
Checking accuracy on Training set after Epoch: 0
Got 270 / 6099 with accuracy of 4.43
Checking accuracy on Validation set after Epoch: 0
Got 51 / 2033 with accuracy of 2.51
On epoch 1
On epoch 2
On epoch 3
On epoch 4
On epoch 5
Checking accuracy on Training set after Epoch: 5
Got 456 / 6099 with accuracy of 7.48
Checking accuracy on Validation set after Epoch: 5
Got 91 / 2033 with accuracy of 4.48
On epoch 6
On epoch 7
On epoch 8
On epoch 9
On epoch 10
Checking accuracy on Training set after Epoch: 10
Got 635 / 6099 with accuracy of 10.41
Checking accuracy on Validation set after Epoch: 10
Got 110 / 2033 with accuracy of 5.41
On epoch 11
On epoch 12
On epoch 13
On epoch 14
On epoch 15
Checking accuracy on Training set after Epoch: 15
Got 808 / 6099 with accuracy of 13.25
Checking accuracy on Validation set after Epoch: 15
Got 137 / 2033 with accuracy of 6.74
On epoch 16
On epoch 17
On epoch 18
On epoch 19
On epoch 20
Checking accuracy on Training set after Epoch: 20
Got 934 

In [ ]:
# Посмотреть в видео варианты для улучшения качества
Sheduler на cosineannealinglr
batchsize
autoaugment